# Profitable App Profiles for the App Store and Google Play Markets

We try to find a profitable free app profile that will suit both the App Store and Google Play markets. For that task, we will be using two data sets - [one](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps) containing data about around 7200 App Store applications, and the [second](https://www.kaggle.com/lava18/google-play-store-apps) one containing data about around 10 000 Google Play applications. 

My goal in this guided project is to further my data analysis skills with some hands-on problems. 

First, let's start by opening our data sets, loading them into lists and then printing the first few rows of each data set to make sure they've been loaded correctly. Let's also count the amount of columns and rows to make sure that we've loaded all data into our lists. 

In [1]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]  
    
    for row in dataset_slice:
        print(row, '\n')

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))
        print('\n')

In [2]:
def read_data(dataset_path): 
    from csv import reader
    open_dataset = open(dataset_path, encoding='utf8')
    read_dataset = reader(open_dataset)
    list_dataset = list(read_dataset) 
    
    return list_dataset

In [3]:
appstore_dataset = read_data('E:\Jupyter\Profitable App Profiles for App Store and Google Play\AppleStore.csv')
googleplay_dataset = read_data('E:\Jupyter\Profitable App Profiles for App Store and Google Play\googleplaystore.csv')

explore_data(appstore_dataset, 0, 5, True)
explore_data(googleplay_dataset, 0, 5, True)

['', 'id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic'] 

['1', '281656475', 'PAC-MAN Premium', '100788224', 'USD', '3.99', '21292', '26', '4', '4.5', '6.3.5', '4+', 'Games', '38', '5', '10', '1'] 

['2', '281796108', 'Evernote - stay organized', '158578688', 'USD', '0', '161065', '26', '4', '3.5', '8.2.2', '4+', 'Productivity', '37', '5', '23', '1'] 

['3', '281940292', 'WeatherBug - Local Weather, Radar, Maps, Alerts', '100524032', 'USD', '0', '188583', '2822', '3.5', '4.5', '5.0.0', '4+', 'Weather', '37', '5', '3', '1'] 

['4', '282614216', 'eBay: Best App to Buy, Sell, Save! Online Shopping', '128512000', 'USD', '0', '262241', '649', '4', '4.5', '5.10.0', '12+', 'Shopping', '37', '5', '9', '1'] 

Number of rows: 7198
Number of columns: 17


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 

As we can see above, our lists seem to be loaded correctly. Now, let's assume that our app has to be free and directed towards english-speaking audience. We need to clean our data sets by removing wrong data and paid or non-english apps and then find columns that might help us with our analysis.

The Google Play data set has a dedicated discussion section, where one of the [discussions](https://www.kaggle.com/lava18/google-play-store-apps/discussion/66015) mentions an error at entry 10472(10473 with header included). Let's print that row and see if it really is incorrect. 

In [4]:
print(googleplay_dataset[10473])

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


It is indeed incorrect. What we can do now is to just simply remove that row with `del` statement and print row 10473 again.

In [5]:
del googleplay_dataset[10473]
print(googleplay_dataset[10473])

['osmino Wi-Fi: free WiFi', 'TOOLS', '4.2', '134203', '4.1M', '10,000,000+', 'Free', '0', 'Everyone', 'Tools', 'August 7, 2018', '6.06.14', '4.4 and up']


Now that we know there are no more errors in our data, we should check our data sets for duplicates by writing a simple function that will check apps' names and create a list of unique apps and a list of duplicates. Note that our data sets are built in a different order, so we need to add `name_index` argument to our function, which will let us work with both data sets using only one function. 

In [6]:
def find_duplicates(dataset, name_index):
    duplicate_apps = []
    unique_apps = []
    
    for row in dataset[1:]:
        if row[0] in unique_apps:
            duplicate_apps.append(row[name_index])
        else:
            unique_apps.append(row[name_index])
            
    print('There are ' + str(len(duplicate_apps)) + ' duplicate apps in the data set')
    len(duplicate_apps)
    
    return len(duplicate_apps)

In [7]:
android_duplicates = find_duplicates(googleplay_dataset, 0)
appstore_duplicates = find_duplicates(appstore_dataset, 2)

There are 1181 duplicate apps in the data set
There are 0 duplicate apps in the data set


We can see there are quite a few duplicates in our android data set that we will need to remove. Since we can't do that randomly, we have to find a criterion which will make our data the most accurate. In this case, we can judge the relevance of our data by the amount of user reviews(the latest app data has the most reviews). 

In [8]:
def remove_duplicates_android(dataset):
    reviews_max = {}
    
    for row in dataset[1:]:
        if row[0] in reviews_max:
            if row[3] > reviews_max[row[0]]:
                reviews_max[row[0]] = row[3]
        else:
            reviews_max[row[0]] = row[3]
            
    print('Expected number of unique android apps: ', len(dataset) - 1 -android_duplicates)
    print('Our number of unique android apps: ', len(reviews_max))
    clean_data = []
    clean_data.append(dataset[0])
    added_apps = []
    
    for row in dataset[1:]:
        if row[3] == reviews_max[row[0]] and row[0] not in added_apps:
            clean_data.append(row)
            added_apps.append(row[0])
            
    return clean_data

In the code above, we created a dictionary to hold the unique names of our apps and their highest amount of reviews. It allowed us later to create two empty lists - for cleaned data and for the names of apps that have already been put into that list. 

Then, we simply just looped through our data set(excluding header) and checked if the current app's number of reviews equals its highest amount possible saved in the `reviews_max` and if its name has already been added to our control list called `added_apps`, that ensures no duplicates are put into `clean_data` list. 

To make sure that our function works correctly, we also checked if the length of cleaned list equals the expected number of unique apps. 

In [9]:
googleplay_dataset = remove_duplicates_android(googleplay_dataset)

Expected number of unique android apps:  9659
Our number of unique android apps:  9659


We can finally move on to the next problem - removing non-english apps. To do that, we should know that each character we use has its own corresponding number, which we can get by using `ord()` function. According to ASCII system, the characters commonly used in english texts are numbered 0 to 127. Since some english apps might use non-english characters, we will assume that non-english app is an app that contains more than 3 non-english characters in its name. We can clean non-english apps from our data sets by checking each row's name column for characters with assigned number outside of our 0-127 range. To do that, we will create a function that iterates through our lists and then removes every app that contains more than 3 non-english characters. 

In [10]:
def remove_non_english(dataset, name_index):
    clean_data = []
    
    for row in dataset:
        non_english_chars = 0
        for character in row[name_index]:
            if ord(character) > 127:
                non_english_chars += 1
        if non_english_chars <= 3:
            clean_data.append(row)
            
    return clean_data

Let's create some simple and short test lists to check if our functions work correctly.

In [11]:
test_list_android = [['Instagram'], ['爱奇艺PPS -《欢乐颂2》电视剧热播'], ['Docs To Go™ Free Office Suite'], ['Instachat 😜']]
test_list_appstore = [['','','Instagram'], ['','','爱奇艺PPS -《欢乐颂2》电视剧热播'], ['','','Docs To Go™ Free Office Suite'], ['','','Instachat 😜']]

test_list_android = remove_non_english(test_list_android, 0)
test_list_appstore = remove_non_english(test_list_appstore, 2)
print(test_list_android, '\n', test_list_appstore)

[['Instagram'], ['Docs To Go™ Free Office Suite'], ['Instachat 😜']] 
 [['', '', 'Instagram'], ['', '', 'Docs To Go™ Free Office Suite'], ['', '', 'Instachat 😜']]


As we can see, our cleaning functions got rid of an app named **爱奇艺PPS -《欢乐颂2》电视剧热播**, but left **Instachat 😜** and **Docs To Go™ Free Office Suite**, both of which also contain non-english characters. Now that we've confirmed that everything works, we can remove non-english apps from our data sets. 

In [12]:
googleplay_dataset = remove_non_english(googleplay_dataset, 0)
appstore_dataset = remove_non_english(appstore_dataset, 2)
print('Remaining number of android apps: ', len(googleplay_dataset))
print('Remaining number of ios apps: ', len(appstore_dataset))

Remaining number of android apps:  9615
Remaining number of ios apps:  6184


The last thing we have to do to clean our data is removing any non-free apps. We can see that in both data sets prices are strings and the price of each free app is a string `'0'`. It allows us to iterate through our data sets and to assign free apps(apps containing `'0'` in price olumn) to a new list. 

In [13]:
def remove_non_free(dataset, price_index):
    clean_data = []
    clean_data.append(dataset[0])
    
    for row in dataset[1:]:
        if row[price_index] == '0':
            clean_data.append(row)
            
    return clean_data

In [14]:
googleplay_dataset = remove_non_free(googleplay_dataset, 7)
appstore_dataset = remove_non_free(appstore_dataset, 5)
print('Number of free android apps: ', len(googleplay_dataset))
print('Number of free ios apps: ', len(appstore_dataset))

Number of free android apps:  8863
Number of free ios apps:  3223


As we can see, after the final cleaning we're left with 8862 Android apps and 3222 iOS apps. We can now move on to finding the most suitable app profile for our analysis. Our app is going to be free, so we need to determine the kinds of apps that are likely to attract the most users. 

Our validation strategy for an app idea is made of three steps:
- We create a minimal version of our app and add it to Google Play
- If the app gets a good response we develop it futher
- If the app is profitable after six months, we build an iOS version and add it to App Store

Since our goal is to find an app that will perform well on Google Play and App Store, we need to find app profiles that are successful on both markets. We can start our analysis by finding the most popular genres on both platforms. To do that, we will write a function that creates an empty dictionary, fills it with genres listed in our data sets and if the app's genre already exists in our dictionary, it will increase our genre's frequency by 1. Then, we convert these values to percentages and return our dictionary sorted in a descending order. 

In [15]:
def freq_table(dataset, index):
    freq_dictionary = {}
    total = 0 
    
    for row in dataset[1:]:
        if row[index] in freq_dictionary:
            freq_dictionary[row[index]] += 1
        else:
            freq_dictionary[row[index]] = 1
        
    for key in freq_dictionary:
        freq_dictionary[key] = round(freq_dictionary[key] / len(dataset[1:]) * 100, 2)
        
    return sorted(freq_dictionary.items(), key=lambda x:x[1], reverse=True)

def display_table(dataset):
    for genre in dataset:
        print(genre)

In [16]:
appstore_genre_freq = freq_table(appstore_dataset, 12)
display_table(appstore_genre_freq)

('Games', 58.16)
('Entertainment', 7.88)
('Photo & Video', 4.97)
('Education', 3.66)
('Social Networking', 3.29)
('Shopping', 2.61)
('Utilities', 2.51)
('Sports', 2.14)
('Music', 2.05)
('Health & Fitness', 2.02)
('Productivity', 1.74)
('Lifestyle', 1.58)
('News', 1.33)
('Travel', 1.24)
('Finance', 1.12)
('Weather', 0.87)
('Food & Drink', 0.81)
('Reference', 0.56)
('Business', 0.53)
('Book', 0.43)
('Navigation', 0.19)
('Medical', 0.19)
('Catalogs', 0.12)


As we can see, App Store is heavily dominated(58.16%) by `Games`. The next most popular genres are `Entertainment`- 7.88%, `Photo & Video`- 4.97%, `Education`- 3.66% and `Social Networking` - 3.29%. These five genres represent 77.96% of all apps on App Store. We can conclude that App Store users tend to use apps that let them relax or kill time. Practical apps have a pretty low representation on this market, but that doesn't necessarily mean they're not popular.

We want to develop our free English app for both iOS and Android, so we can't recommend an app profile yet, as the popularity distribution might be different on Google Play. 

In [17]:
googleplay_genre_freq = freq_table(googleplay_dataset, 9)
display_table(googleplay_genre_freq)

('Tools', 8.44)
('Entertainment', 6.07)
('Education', 5.35)
('Business', 4.59)
('Lifestyle', 3.89)
('Productivity', 3.89)
('Finance', 3.7)
('Medical', 3.52)
('Sports', 3.46)
('Personalization', 3.32)
('Communication', 3.24)
('Action', 3.1)
('Health & Fitness', 3.08)
('Photography', 2.95)
('News & Magazines', 2.8)
('Social', 2.66)
('Travel & Local', 2.32)
('Shopping', 2.25)
('Books & Reference', 2.14)
('Simulation', 2.04)
('Dating', 1.86)
('Arcade', 1.85)
('Video Players & Editors', 1.77)
('Casual', 1.75)
('Maps & Navigation', 1.4)
('Food & Drink', 1.24)
('Puzzle', 1.13)
('Racing', 0.99)
('Libraries & Demo', 0.94)
('Role Playing', 0.94)
('Auto & Vehicles', 0.93)
('Strategy', 0.91)
('House & Home', 0.82)
('Weather', 0.8)
('Events', 0.71)
('Adventure', 0.68)
('Comics', 0.61)
('Art & Design', 0.6)
('Beauty', 0.6)
('Parenting', 0.5)
('Card', 0.44)
('Casino', 0.43)
('Trivia', 0.42)
('Educational;Education', 0.39)
('Board', 0.37)
('Educational', 0.37)
('Education;Education', 0.34)
('Word', 0.

Our android data set's `genre` column seems to be very detailed, which doesn't bring us any clear benefits and can make our work more inconvenient, so we will focus on the `category` column instead. 

In [18]:
googleplay_category_freq = freq_table(googleplay_dataset, 1)
display_table(googleplay_category_freq)

('FAMILY', 18.93)
('GAME', 9.69)
('TOOLS', 8.45)
('BUSINESS', 4.59)
('LIFESTYLE', 3.9)
('PRODUCTIVITY', 3.89)
('FINANCE', 3.7)
('MEDICAL', 3.52)
('SPORTS', 3.4)
('PERSONALIZATION', 3.32)
('COMMUNICATION', 3.24)
('HEALTH_AND_FITNESS', 3.08)
('PHOTOGRAPHY', 2.95)
('NEWS_AND_MAGAZINES', 2.8)
('SOCIAL', 2.66)
('TRAVEL_AND_LOCAL', 2.34)
('SHOPPING', 2.25)
('BOOKS_AND_REFERENCE', 2.14)
('DATING', 1.86)
('VIDEO_PLAYERS', 1.79)
('MAPS_AND_NAVIGATION', 1.4)
('FOOD_AND_DRINK', 1.24)
('EDUCATION', 1.17)
('ENTERTAINMENT', 0.96)
('LIBRARIES_AND_DEMO', 0.94)
('AUTO_AND_VEHICLES', 0.93)
('HOUSE_AND_HOME', 0.82)
('WEATHER', 0.8)
('EVENTS', 0.71)
('PARENTING', 0.65)
('ART_AND_DESIGN', 0.64)
('COMICS', 0.62)
('BEAUTY', 0.6)


We can easily notice that unlike App Store, Google Play isn't as heavily dominated by one genre. The two most popular categories are `Family` with 18.93% and `Games` with 9.69%. Right behind them are `Tools` with 8.45%, `Business` with 4.59% and `Lifestyle` with 3.9%, meaning that although the most popular categories are still relaxing time killers, the practical apps are way more common on Google Play than they are on App Store, making it a more balanced market. 

As we have said before, we can't recommend an app profile basing only on apps distribution by genre. What we need to find now is how popular each genre is on average. To do that, we will create a simple function that calculates the average number of installs on Google Play and the average number of user reviews on App Store. Google Play data set's `Installs` column is filled with open-ended strings(e.g. 100,000+, 1,000,000+) that need to be converted to integers for us to calculate the genre's average number of users. We can do that by removing all commas and pluses from our strings.

In [19]:
def how_popular_genre(dataset, genre_index, popularity_index):
    popularity_dictionary = {}
    freq_dictionary = {}
    
    for row in dataset[1:]:
        if row[genre_index] in popularity_dictionary:
            nr_of_users = row[popularity_index]
            nr_of_users = nr_of_users.replace(',', '')
            nr_of_users = nr_of_users.replace('+', '')
            freq_dictionary[row[genre_index]] += 1
            popularity_dictionary[row[genre_index]] += float(nr_of_users)
        else:
            nr_of_users = row[popularity_index]
            nr_of_users = nr_of_users.replace(',', '')
            nr_of_users = nr_of_users.replace('+', '')
            freq_dictionary[row[genre_index]] = 1
            popularity_dictionary[row[genre_index]] = float(nr_of_users)
            
    for genre in popularity_dictionary:
        popularity_dictionary[genre] = round(popularity_dictionary[genre] / freq_dictionary[genre], 2)   
        
    return sorted(popularity_dictionary.items(), key=lambda x:x[1], reverse=True)

In [20]:
appstore_genre_popularity = how_popular_genre(appstore_dataset, 12, 6)
display_table(appstore_genre_popularity)

('Navigation', 86090.33)
('Reference', 74942.11)
('Social Networking', 71548.35)
('Music', 57326.53)
('Weather', 52279.89)
('Book', 39758.5)
('Food & Drink', 33333.92)
('Finance', 31467.94)
('Photo & Video', 28441.54)
('Travel', 28243.8)
('Shopping', 26919.69)
('Health & Fitness', 23298.02)
('Sports', 23008.9)
('Games', 22788.67)
('News', 21248.02)
('Productivity', 21028.41)
('Utilities', 18684.46)
('Lifestyle', 16485.76)
('Entertainment', 14029.83)
('Business', 7491.12)
('Education', 7003.98)
('Catalogs', 4004.0)
('Medical', 612.0)


We can probably dismiss the `Social Networking` and `Music` genre since they require a lot of work and resources to become successful. `Weather` also won't be suitable for our app because people spend very short time using such apps and thus, it wouldn't bring us much money from ads and would require a very accurate weather data for people to use it. With them out of our way, let's explore the `Navigation`, `Reference` and `Book` genres.

In [21]:
def most_popular_apps_in_genre(dataset, genre_index, popularity_index, name_index, genre):
    apps_list = []
    
    for row in dataset[1:]:
        if row[genre_index] == genre:
              apps_list.append(row)
                
    apps_list = sorted(apps_list, key=lambda x: float(x[popularity_index]), reverse=True)
    
    for row in apps_list[:20]:
        print(row[name_index], ' : ', row[popularity_index])
    print('\n')

In [22]:
most_popular_apps_in_genre(appstore_dataset, 12, 6, 2, 'Navigation')
most_popular_apps_in_genre(appstore_dataset, 12, 6, 2, 'Reference')
most_popular_apps_in_genre(appstore_dataset, 12, 6, 2, 'Book')  

Waze - GPS Navigation, Maps & Real-time Traffic  :  345046
Google Maps - Navigation & Transit  :  154911
Geocaching®  :  12811
CoPilot GPS – Car Navigation & Offline Maps  :  3582
ImmobilienScout24: Real Estate Search in Germany  :  187
Railway Route Search  :  5


Bible  :  985920
Dictionary.com Dictionary & Thesaurus  :  200047
Dictionary.com Dictionary & Thesaurus for iPad  :  54175
Google Translate  :  26786
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran  :  18418
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition  :  17588
Merriam-Webster Dictionary  :  16849
Night Sky  :  12122
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE)  :  8535
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools  :  4693
GUNS MODS for Minecraft PC Edition - Mods Tools  :  1497
Guides for Pokémon GO - Pokemon GO News and Cheats  :  826
WWDC  :  762
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraf

Even though `Navigation` and `Book` genres have high average numbers of reviews, we can see that those averages are highly influenced by 2-3 most popular apps, while the rest barely has any presence on the market. It leaves us with `Reference` genre, which seems to show some potential. Despite its average also being influenced by top 2 apps, the genre itself has a much more balanced landscape. 

This might actually be a perfect idea for our app, since users frequently spend a lot of time in apps like 'Bible', 'Dictionary' or various Wikis, allowing us to get more profit from ads. We could create a wiki-like app, containing guides for the most popular games. The dynamics of the gaming market would give us a steady stream of new content and users, making it more likely for our app to be profitable in a long run.

Alternatively, we could copy `Bible` and `Quran` ideas and make an app about Hindu/Buddhist holy books, since this type of an app usually does well.

Let's see if this idea would be good for the Google Play market.

In [23]:
googleplay_genre_popularity = how_popular_genre(googleplay_dataset, 1, 5)
display_table(googleplay_genre_popularity)

('COMMUNICATION', 38456119.17)
('VIDEO_PLAYERS', 24727872.45)
('SOCIAL', 23253652.13)
('PHOTOGRAPHY', 17805627.64)
('PRODUCTIVITY', 16787331.34)
('GAME', 15560965.6)
('TRAVEL_AND_LOCAL', 13984077.71)
('ENTERTAINMENT', 11640705.88)
('TOOLS', 10682301.03)
('NEWS_AND_MAGAZINES', 9549178.47)
('BOOKS_AND_REFERENCE', 8767811.89)
('SHOPPING', 7036877.31)
('PERSONALIZATION', 5201482.61)
('WEATHER', 5074486.2)
('HEALTH_AND_FITNESS', 4188821.99)
('MAPS_AND_NAVIGATION', 4056941.77)
('FAMILY', 3694276.33)
('SPORTS', 3638640.14)
('ART_AND_DESIGN', 1986335.09)
('FOOD_AND_DRINK', 1924897.74)
('EDUCATION', 1820673.08)
('BUSINESS', 1712290.15)
('LIFESTYLE', 1437816.27)
('FINANCE', 1387692.48)
('HOUSE_AND_HOME', 1331540.56)
('DATING', 854028.83)
('COMICS', 817657.27)
('AUTO_AND_VEHICLES', 647317.82)
('LIBRARIES_AND_DEMO', 638503.73)
('PARENTING', 542603.62)
('BEAUTY', 513151.89)
('EVENTS', 253542.22)
('MEDICAL', 120616.49)


The situation here is fairly similar to the App Store market. The most popular genres(`COMMUNICATION`, `VIDEO_PLAYERS` and `SOCIAL`) require a lot of work and their numbers are pumped by big apps like Messenger, Youtube and Facebook. Thankfully, the `BOOKS_AND_REFERENCE` genre seems to be fairly popular too. Let's explore it in more detail.

In [24]:
for row in googleplay_dataset[1:]:
    if row[1] == 'BOOKS_AND_REFERENCE' and (row[5] == '1,000,000+'
                                            or row[5] == '5,000,000+'
                                            or row[5] == '10,000,000+'
                                            or row[5] == '50,000,000+'
                                            or row[5] == '100,000,000+'):
        print(row[0], ' : ', row[5])

Wikipedia  :  10,000,000+
Cool Reader  :  10,000,000+
Book store  :  1,000,000+
FBReader: Favorite Book Reader  :  10,000,000+
Free Books - Spirit Fanfiction and Stories  :  1,000,000+
AlReader -any text book reader  :  5,000,000+
FamilySearch Tree  :  1,000,000+
Cloud of Books  :  1,000,000+
ReadEra – free ebook reader  :  1,000,000+
Ebook Reader  :  5,000,000+
Read books online  :  5,000,000+
eBoox: book reader fb2 epub zip  :  1,000,000+
All Maths Formulas  :  1,000,000+
Ancestry  :  5,000,000+
HTC Help  :  10,000,000+
Moon+ Reader  :  10,000,000+
English-Myanmar Dictionary  :  1,000,000+
Golden Dictionary (EN-AR)  :  1,000,000+
All Language Translator Free  :  1,000,000+
Bible  :  100,000,000+
Amazon Kindle  :  100,000,000+
Aldiko Book Reader  :  10,000,000+
Wattpad 📖 Free Books  :  100,000,000+
Dictionary - WordWeb  :  5,000,000+
50000 Free eBooks & Free AudioBooks  :  5,000,000+
Al-Quran (Free)  :  10,000,000+
Al Quran Indonesia  :  10,000,000+
Al'Quran Bahasa Indonesia  :  10,00

Google Play on the other hand seems to be dominated by ebooks software, dictionaries and libraries. Religion books focused apps are also quite popular. The other thing that we can notice is a small number of wiki apps.

The above observation indicates that our app about Hindu/Buddhist religious books has a chance to become profitable on both markets. If our app does well, we could also consider making it a bilingual and adding hindi language, giving us even more exposure.

Our wiki app for various games also doesn't seem like a bad idea since there isn't much competition on the Android market. Since the most popular games tend to be the multiplayer ones, if we do some serious research and write useful guides, we might attract a lot of players wanting to improve at the game they like. This idea, compared to our other one, is harder to realize, but would give us a steadier stream of users. 

# Conclusions
We analyzed data about App Store and Google Play markets and tried to find an app profile that could be profitable for both markets.

We came to the conclusion that apps focusing on religious books might have a high chance of attracting a lot of users. Their negative side is that there's not a lot of further developement we can do after their initial version. It shouldn't hinder the new users stream by a lot since there are new religion practitioners everyday. 

The games wiki app on the other hand is a more risky idea, but we could develop it for many years, never suffering from the lack of new content or ideas. If our guides were well-made, the users would keep coming back to our app whenever they started playing a new game. This would give us a constant stream of users, who would spend a significant amount of time reading our guides, bringing us more profit from ads. 